Image Processing:

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def custom_convolution(image, kernel):
    img_height, img_width, channels = image.shape
    k_height, k_width = kernel.shape
    pad_h, pad_w = k_height // 2, k_width // 2

    padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant')

    output = np.zeros_like(image)

    for y in range(img_height):
        for x in range(img_width):
            for c in range(channels):
                output[y, x, c] = np.sum(
                    padded_image[y:y+k_height, x:x+k_width, c] * kernel
                )

    return np.clip(output, 0, 255).astype(np.uint8)

def apply_motion_blur(image_path, kernel_size=15):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Create a horizontal motion blur kernel
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
    kernel /= kernel_size  # Normalize the kernel

    # Apply custom convolution instead of OpenCV filter
    blurred = custom_convolution(image, kernel)

    # Display original and blurred images
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    ax[1].imshow(blurred)
    ax[1].set_title("Motion Blurred Image")
    ax[1].axis("off")

    plt.show()
    return blurred

# Example usage
image_path = "original_image.jpg"  # Replace with your image file path
apply_motion_blur(image_path)


Filter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Generate a sample signal
fs = 1000  # Sampling frequency (Hz)
t = np.linspace(0, 1, fs, endpoint=False)  # 1 second of data
signal = np.sin(2 * np.pi * 10 * t) + 0.5 * np.sin(2 * np.pi * 50 * t)  # 10 Hz + 50 Hz components
noise = np.random.normal(0, 0.3, signal.shape)  # Additive Gaussian noise
noisy_signal = signal

# Define Butterworth filter function
def butter_filter(data, cutoff, fs, filter_type, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype=filter_type, analog=False)
    return filtfilt(b, a, data)

# Apply Low-Pass Filter (removes high-frequency noise)
lpf_signal = butter_filter(noisy_signal, 20, fs, 'low')

# Apply High-Pass Filter (removes low-frequency components)
hpf_signal = butter_filter(noisy_signal, 20, fs, 'high')

# Plot results
plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 1)
plt.plot(t, noisy_signal, label='Noisy Signal', color='gray')
plt.title('Noisy Signal (10Hz + 50Hz + Noise)')
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(t, lpf_signal, label='Low-Pass Filtered (Removes >20Hz)', color='blue')
plt.title('Low-Pass Filter Effect')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(t, hpf_signal, label='High-Pass Filtered (Removes <20Hz)', color='red')
plt.title('High-Pass Filter Effect')
plt.legend()

plt.tight_layout()
plt.show()


Power Spectrum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft

# Generate a sample signal
fs = 1000  # Sampling frequency (Hz)
t = np.linspace(0, 1, fs, endpoint=False)  # Time vector
signal = np.sin(2 * np.pi * 50 * t) + 0.5 * np.sin(2 * np.pi * 120 * t)  # 50Hz + 120Hz

# Compute Fourier Transform
X = fft(signal)
frequencies = np.fft.fftfreq(len(X), 1/fs)
power_spectrum = np.abs(X)**2

# Plot Power Spectrum
plt.figure(figsize=(8, 4))
plt.plot(frequencies[:fs//2], power_spectrum[:fs//2], color='blue')
plt.title("Power Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid()
plt.show()


Correlation:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate, convolve

def generate_signals(N):
    # Step signal: Jumps from 0 to 1 at t=0
    step_signal = np.zeros(N)
    step_signal[N//4:] = 1  # Jump at 1/4th of total length

    # Sawtooth signal: Increases linearly then drops to 0
    sawtooth_signal = np.linspace(1, 0, N//2)
    sawtooth_signal = np.concatenate((np.zeros(N//2),sawtooth_signal))

    return step_signal, sawtooth_signal

def process_signals():
    N = 100  # Signal length
    step_signal, sawtooth_signal = generate_signals(N)

    # Convolution
    conv_result = convolve(step_signal, sawtooth_signal, mode='full')

    # Auto-correlation
    auto_corr_step = correlate(step_signal, step_signal, mode='full')
    auto_corr_sawtooth = correlate(sawtooth_signal, sawtooth_signal, mode='full')

    # Cross-correlation
    cross_corr = correlate(step_signal, sawtooth_signal, mode='full')

    # Plot results
    plt.figure(figsize=(12, 8))

    # Original signals
    plt.subplot(3, 2, 1)
    plt.plot(step_signal, label='Step Signal')
    plt.title("Step Signal")
    plt.legend()

    plt.subplot(3, 2, 2)
    plt.plot(sawtooth_signal, label='Sawtooth Signal')
    plt.title("Sawtooth Signal")
    plt.legend()

    # Convolution
    plt.subplot(3, 2, 3)
    plt.plot(conv_result, label='Convolution')
    plt.title("Convolution of Step & Sawtooth Signals")
    plt.legend()

    # Auto-correlation
    plt.subplot(3, 2, 4)
    plt.plot(auto_corr_step, label='Auto-corr Step')
    plt.plot(auto_corr_sawtooth, label='Auto-corr Sawtooth', linestyle='dashed')
    plt.title("Auto-correlation")
    plt.legend()

    # Cross-correlation
    plt.subplot(3, 2, 5)
    plt.plot(cross_corr, label='Cross-correlation')
    plt.title("Cross-correlation between Step & Sawtooth")
    plt.legend()

    plt.tight_layout()
    plt.show()

process_signals()